# 🎙️ PersonaPlex Demo — Proof of Concept

**Proyecto:** PersonaPlex Callbot (Fase 1)

Este notebook prueba PersonaPlex en Google Colab con GPU T4.

### Antes de empezar:
1. ✅ Acepta la licencia del modelo: [nvidia/personaplex-7b-v1](https://huggingface.co/nvidia/personaplex-7b-v1)
2. ✅ Ten tu HuggingFace token listo
3. ✅ Asegúrate de estar en Runtime → GPU T4

---

## 1️⃣ Verificar GPU

In [ ]:
!nvidia-smi
import torch
print(f"\n✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_mem / 1024**3:.1f} GB")

## 2️⃣ Instalar dependencias del sistema

In [ ]:
!apt-get update -qq && apt-get install -y -qq libopus-dev > /dev/null 2>&1
print("✅ libopus-dev instalado")

## 3️⃣ Clonar el repositorio e instalar PersonaPlex

In [ ]:
!git clone https://github.com/NVIDIA/personaplex.git
%cd personaplex
!pip install -q moshi/.
!pip install -q accelerate  # Needed for --cpu-offload
print("\n✅ PersonaPlex instalado")

## 4️⃣ Autenticación con HuggingFace

Ingresa tu token de HuggingFace (necesario para descargar los weights).

In [ ]:
import os
from getpass import getpass

# Opción 1: Ingreso manual
hf_token = getpass("🔑 Ingresa tu HuggingFace Token: ")
os.environ["HF_TOKEN"] = hf_token

# Verificar
print(f"✅ Token configurado ({len(hf_token)} chars)")

## 5️⃣ Test Offline — Modo Asistente

Primer test: usar el audio de prueba incluido con la voz NATF2 (femenina natural).

⚠️ Usamos `--cpu-offload` porque el T4 tiene 16GB VRAM y el modelo pesa ~14GB en FP16.

In [ ]:
%%time
!python -m moshi.offline \
    --voice-prompt "NATF2.pt" \
    --input-wav "assets/test/input_assistant.wav" \
    --seed 42424242 \
    --output-wav "output_assistant.wav" \
    --output-text "output_assistant.json" \
    --cpu-offload

print("\n✅ Audio generado: output_assistant.wav")

### Reproducir resultado

In [ ]:
from IPython.display import Audio, display, JSON
import json

print("🎧 Audio de entrada (lo que 'escuchó' PersonaPlex):")
display(Audio("assets/test/input_assistant.wav"))

print("\n🎙️ Respuesta de PersonaPlex:")
display(Audio("output_assistant.wav"))

print("\n📝 Transcripción de la respuesta:")
with open("output_assistant.json") as f:
    transcript = json.load(f)
    print(json.dumps(transcript, indent=2))

## 6️⃣ Test Offline — Modo Customer Service

Segundo test: modo servicio al cliente con voz masculina NATM1 y un prompt de rol.

In [ ]:
%%time
!python -m moshi.offline \
    --voice-prompt "NATM1.pt" \
    --text-prompt "$(cat assets/test/prompt_service.txt)" \
    --input-wav "assets/test/input_service.wav" \
    --seed 42424242 \
    --output-wav "output_service.wav" \
    --output-text "output_service.json" \
    --cpu-offload

print("\n✅ Audio generado: output_service.wav")

In [ ]:
print("🎧 Audio de entrada (cliente):")
display(Audio("assets/test/input_service.wav"))

print("\n🎙️ Respuesta de PersonaPlex (agente):")
display(Audio("output_service.wav"))

print("\n📝 Transcripción:")
with open("output_service.json") as f:
    transcript = json.load(f)
    print(json.dumps(transcript, indent=2))

## 7️⃣ Probar diferentes voces

Iteramos sobre varias voces para comparar calidad y estilo.

In [ ]:
voices_to_test = ["NATF0", "NATF2", "NATM0", "NATM2", "VARF1", "VARM1"]

import subprocess, json
from IPython.display import Audio, display

results = {}

for voice in voices_to_test:
    print(f"\n{'='*50}")
    print(f"🎤 Probando voz: {voice}")
    print(f"{'='*50}")
    
    out_wav = f"output_voice_{voice}.wav"
    out_json = f"output_voice_{voice}.json"
    
    result = subprocess.run([
        "python", "-m", "moshi.offline",
        "--voice-prompt", f"{voice}.pt",
        "--input-wav", "assets/test/input_assistant.wav",
        "--seed", "42424242",
        "--output-wav", out_wav,
        "--output-text", out_json,
        "--cpu-offload"
    ], capture_output=True, text=True)
    
    if result.returncode == 0:
        print(f"✅ {voice} completado")
        display(Audio(out_wav))
        
        with open(out_json) as f:
            transcript = json.load(f)
            results[voice] = transcript
            print(f"📝 {json.dumps(transcript, indent=2)[:300]}")
    else:
        print(f"❌ Error con {voice}: {result.stderr[:200]}")

print(f"\n\n🏁 Voces probadas: {len(results)}/{len(voices_to_test)}")

## 8️⃣ Probar prompt personalizado (Callbot)

Simulamos un escenario de callbot tipo OnBotGo.

In [ ]:
# Prompt personalizado estilo callbot
custom_prompt = """You work for TechSupport Pro which is a technical support service and your name is Sarah. \
Information: You help customers troubleshoot internet connectivity issues. \
Common solutions: restart router (wait 30 seconds), check cable connections, \
run speed test at speedtest.net. If issue persists, schedule technician visit \
(next available: tomorrow 2-4 PM or Thursday 9-11 AM). \
Service costs: Basic plan $29.99/month, Premium $49.99/month with priority support."""

# Guardar prompt
with open("custom_prompt.txt", "w") as f:
    f.write(custom_prompt)

print("Prompt guardado ✅")
print(f"\n📋 Prompt:\n{custom_prompt}")

In [ ]:
%%time
# Usar el audio de servicio como input (simula un cliente llamando)
!python -m moshi.offline \
    --voice-prompt "NATF2.pt" \
    --text-prompt "$(cat custom_prompt.txt)" \
    --input-wav "assets/test/input_service.wav" \
    --seed 42424242 \
    --output-wav "output_callbot.wav" \
    --output-text "output_callbot.json" \
    --cpu-offload

print("\n✅ Callbot test completado")

In [ ]:
print("🎙️ Respuesta del callbot:")
display(Audio("output_callbot.wav"))

print("\n📝 Transcripción:")
with open("output_callbot.json") as f:
    transcript = json.load(f)
    print(json.dumps(transcript, indent=2))

## 9️⃣ Métricas de Rendimiento

Capturamos tiempos y uso de recursos para evaluar viabilidad.

In [ ]:
import time, os, subprocess

print("📊 Benchmark: Latencia de generación offline")
print("="*50)

# Medir tiempo de una generación
start = time.time()
result = subprocess.run([
    "python", "-m", "moshi.offline",
    "--voice-prompt", "NATF2.pt",
    "--input-wav", "assets/test/input_assistant.wav",
    "--seed", "12345",
    "--output-wav", "benchmark_output.wav",
    "--output-text", "benchmark_output.json",
    "--cpu-offload"
], capture_output=True, text=True)
elapsed = time.time() - start

# Duración del audio input
import wave
with wave.open("assets/test/input_assistant.wav") as w:
    input_duration = w.getnframes() / w.getframerate()

with wave.open("benchmark_output.wav") as w:
    output_duration = w.getnframes() / w.getframerate()

print(f"⏱️  Tiempo total: {elapsed:.1f}s")
print(f"🎵 Duración input: {input_duration:.1f}s")
print(f"🎵 Duración output: {output_duration:.1f}s")
print(f"⚡ RTF (Real Time Factor): {elapsed/output_duration:.2f}x")
print(f"   (< 1.0 = más rápido que tiempo real)")

# GPU memory
!nvidia-smi --query-gpu=memory.used,memory.total --format=csv,noheader

print(f"\n{'='*50}")
print("📋 Resumen para PROJECT.md:")
print(f"   GPU: T4 16GB + cpu-offload")
print(f"   RTF: {elapsed/output_duration:.2f}x")
print(f"   Tiempo gen: {elapsed:.1f}s para {output_duration:.1f}s de audio")

## 🔟 (Bonus) Servidor Web Interactivo

⚠️ Solo funciona si tienes **micrófono** disponible (Colab con Chrome).

Esto levanta el Web UI de PersonaPlex con interacción en tiempo real.

In [ ]:
# Descomenta para probar el servidor interactivo
# ⚠️ En Colab free, la sesión puede morir por timeout
# ⚠️ Necesita HTTPS para acceso al micrófono

# import subprocess, threading
# 
# def run_server():
#     subprocess.run([
#         "python", "-m", "moshi.server",
#         "--ssl", "/tmp/ssl_certs",
#         "--cpu-offload"
#     ])
# 
# # Crear dir SSL
# !mkdir -p /tmp/ssl_certs
# 
# # Lanzar en background
# thread = threading.Thread(target=run_server, daemon=True)
# thread.start()
# 
# print("🌐 Servidor lanzado. Accede via:")
# print("   https://localhost:8998")
# print("   (O usa localtunnel/ngrok para acceso remoto)")

---

## ✅ Resultados Fase 1

| Métrica | Valor |
|---------|-------|
| GPU usada | T4 16GB |
| cpu-offload | Sí |
| RTF | *(llenar)* |
| Calidad audio | *(evaluar subjetivamente)* |
| Mejor voz femenina | *(llenar)* |
| Mejor voz masculina | *(llenar)* |
| Adherencia al prompt | *(evaluar)* |
| Solo inglés | ✅ Confirmado |

### Próximos pasos:
- [ ] Probar en Mac mini con personaplex-mlx (Fase 1, tarea 2)
- [ ] Evaluar si RTF < 1.0 es alcanzable en local
- [ ] Diseñar integración con Twilio (Fase 2)